In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score

# === Configuration ===
data_dir    = r"C:\Users\aruna\Documents\projects\code_base\data\training"
num_epochs  = 10
batch_size  = 8
lr          = 1e-4
weight_decay= 1e-5
val_split   = 0.2
device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(os.listdir(data_dir))  # assumes one subfolder per class

# === Transforms & Dataset ===
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

full_dataset = ImageFolder(root=data_dir, transform=transform)
val_size     = int(len(full_dataset) * val_split)
train_size   = len(full_dataset) - val_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4)

# === Model Setup ===
model = models.densenet121(pretrained=True)

# 1) Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# 2) Unfreeze last dense block and classifier
for name, param in model.named_parameters():
    if name.startswith("features.denseblock4") or name.startswith("classifier"):
        param.requires_grad = True

# 3) Replace classifier head
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, num_classes)

model = model.to(device)

# === Loss & Optimizer ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=lr,
    weight_decay=weight_decay
)

# === Training & Validation Loop ===
best_val_acc = 0.0
for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss    += loss.item() * imgs.size(0)
        preds          = outputs.argmax(dim=1)
        train_correct += (preds == labels).sum().item()
        train_total   += labels.size(0)

    train_loss /= train_total
    train_acc   = train_correct / train_total

    # --- Validation ---
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            out  = model(imgs)
            val_preds.extend(out.argmax(dim=1).cpu().tolist())
            val_labels.extend(labels.tolist())

    val_acc = accuracy_score(val_labels, val_preds)

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"— Train: loss={train_loss:.4f}, acc={train_acc:.4f} "
          f"— Val: acc={val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_densenet_finetuned.pth")

# === Save final validation accuracy to text file ===
with open("validation_accuracy.txt", "w") as f:
    f.write(f"Best validation accuracy: {best_val_acc:.4f}\n")

print(f"Training complete. Best val acc: {best_val_acc:.4f}")

C:\Users\aruna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\aruna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10 — Train: loss=1.0111, acc=0.4689 — Val: acc=0.6719
Epoch 2/10 — Train: loss=0.8552, acc=0.5817 — Val: acc=0.5938
Epoch 3/10 — Train: loss=0.7810, acc=0.6712 — Val: acc=0.6953
Epoch 4/10 — Train: loss=0.7294, acc=0.7023 — Val: acc=0.6719
Epoch 5/10 — Train: loss=0.6656, acc=0.6984 — Val: acc=0.6797
Epoch 6/10 — Train: loss=0.5891, acc=0.7471 — Val: acc=0.6875
Epoch 7/10 — Train: loss=0.5755, acc=0.7743 — Val: acc=0.7109
Epoch 8/10 — Train: loss=0.4840, acc=0.8054 — Val: acc=0.6953
Epoch 9/10 — Train: loss=0.4948, acc=0.7802 — Val: acc=0.7812
Epoch 10/10 — Train: loss=0.4051, acc=0.8346 — Val: acc=0.6719
Training complete. Best val acc: 0.7812


In [7]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# === Configuration ===
model_path  = "best_densenet_finetuned.pth"
image_path  = r"C:\Users\aruna\Documents\projects\code_base\data\testing\Non‑Demented\42.png"
device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 1. Load the checkpoint & infer number of classes ===
checkpoint = torch.load(model_path, map_location=device)
# classifier.weight has shape [num_classes, in_features]
num_classes = checkpoint['classifier.weight'].size(0)
print(f"Checkpoint was trained with {num_classes} output classes.")

# === 2. Rebuild the model with that many outputs ===
model = models.densenet121(pretrained=False)           # no ImageNet head
in_feats = model.classifier.in_features                # should be 1024
model.classifier = nn.Linear(in_feats, num_classes)    # match checkpoint
model.load_state_dict(checkpoint)                      # load all weights
model = model.to(device)
model.eval()

# === 3. Define your class names (length must == num_classes) ===
# Replace these with the actual folder names you used when training,
# in the exact same order as ImageFolder assigned them.
class_names = ["Mild", "Moderate", "Non-Demanted"]  # e.g. ["AD","MCI","CN"] for 3 classes

assert len(class_names) == num_classes, "class_names length must match num_classes"

# === 4. Preprocessing transforms (same as training) ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# === 5. Load & preprocess the test image ===
img = Image.open(image_path).convert("RGB")
x   = transform(img).unsqueeze(0).to(device)  # shape [1,3,224,224]

# === 6. Inference ===
with torch.no_grad():
    logits = model(x)                         # [1, num_classes]
    probs  = torch.softmax(logits, dim=1)[0]  # [num_classes]

pred_idx = probs.argmax().item()
print(f"Predicted class: {class_names[pred_idx]}")
print(f"Probabilities: {probs.cpu().numpy()}")


C:\Users\aruna\AppData\Local\Temp\ipykernel_33608\1335622665.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


Checkpoint was trained with 3 output classes.
Predicted class: Non-Demanted
Probabilities: [1.1809778e-01 8.0679858e-04 8.8109547e-01]
